<a href="https://colab.research.google.com/github/marcinwolter/Machine-learning-KISD-2023/blob/main/lecture7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>




#**<font color = "red">Introduction to machine learning</font>**

**Lecture 7**


##**<font color = "green">Generative Adversary Network - GAN</font>**

*19 April 2023*


---

*Marcin Wolter, IFJ PAN*

*e-mail: marcin.wolter@ifj.edu.pl*


---
</center>

#<font color='green'>**Program for today:**


* ###  <font color='red'>Autoencoder: like PCA, but not linear 
* ###  <font color='red'>Reinforcement learning: how to train a robot?
* ###  <font color='red'>Transfer learning: using pretrained networks.


<br>


**As always all slides are here:** 

*https://github.com/marcinwolter/Machine-learning-KISD-2023*

<br>




# **Ideas about your projects???**

# <font color='green'> **Transfer learning**

The reuse of a previously learned model on a new problem is known as transfer learning. It’s particularly popular in deep learning right now since it can train deep neural networks with a small amount of data.

<img src = 'https://editor.analyticsvidhya.com/uploads/444861_2vse-G3FKMT2D59NFWduMA.jpg' width=450px>

The knowledge of an already trained machine learning model is transferred to a different but closely linked problem throughout transfer learning. For example, if you trained a simple classifier to predict whether an image contains a backpack, you could use the model’s training knowledge to identify other objects such as sunglasses.



# **What is Transfer Learning?**
With transfer learning, we basically try to use what we’ve learned in one task to better understand the concepts in another. Weights are being automatically being shifted to a network performing “task A” from a network that performed new “task B.”


<img src = 'https://editor.analyticsvidhya.com/uploads/751191_rsgubd7aTgUdY65KPYATBA.png' width=450px>

So a network trained to recognize a "cat" can be reused to recognize a "dog".


<img src = 'https://editor.analyticsvidhya.com/uploads/499849315476_1592890541_transfer.jpg' width=500px>

Because the model has trained to recognise objects in the earlier levels, we will simply retrain the subsequent layers to understand what distinguishes sunglasses from other objects.

**Option is to employ a model that has already been trained. There are a number of these models out there, so do some research beforehand. The number of layers to reuse and retrain is determined by the task.**

Keras consists of many pre-trained models used in transfer learning, prediction, fine-tuning. 

#<font color='green'> **Summary:**

* <span style="color:#000000">Many new methods were developed recently\.</span>

* <span style="color:#000000">Machine Learning approach becomes to be used not only for classification\, but also for generative tasks\.</span>

* <span style="color:#000000">Each month new application appear\! </span>
